# Production Analysis Kickstart

This notebook summarizes FoF/GalCenter outputs for the four production run directories.

In [ ]:
from pathlib import Path

try:
    import pandas as pd
except Exception:
    pd = None

try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None

WORKDIRS = [
    Path('/home/bkoh/pgalf/work/production_runs/zoom'),
    Path('/home/bkoh/pgalf/work/production_runs/zoom2'),
    Path('/home/bkoh/pgalf/work/production_runs/zoom3'),
    Path('/home/bkoh/pgalf/work/production_runs/zoom4'),
]

WORKDIRS


In [ ]:
def file_size(path):
    return path.stat().st_size if path.exists() else 0

rows = []
for wd in WORKDIRS:
    fof_root = wd / 'FoF_Data'
    fof_dirs = sorted([p for p in fof_root.glob('FoF.*') if p.is_dir()])

    galfind_data_nonempty = 0
    galcenter_nonempty = 0
    fof_halo_nonempty = 0

    for d in fof_dirs:
        step = d.name.split('.')[-1]
        data_file = d / f'GALFIND.DATA.{step}'
        center_file = d / f'GALFIND.CENTER.{step}'
        halo_file = d / f'FoF_halo_cat.{step}'

        if file_size(data_file) > 0:
            galfind_data_nonempty += 1
        if file_size(center_file) > 0:
            galcenter_nonempty += 1
        if file_size(halo_file) > 0:
            fof_halo_nonempty += 1

    rows.append({
        'variant': wd.name,
        'fof_dirs_present': len(fof_dirs),
        'galfind_data_nonempty': galfind_data_nonempty,
        'galcenter_nonempty': galcenter_nonempty,
        'fof_halo_nonempty': fof_halo_nonempty,
    })

if pd is not None:
    df = pd.DataFrame(rows).sort_values('variant').reset_index(drop=True)
    display(df)
else:
    headers = ['variant', 'fof_dirs_present', 'galfind_data_nonempty', 'galcenter_nonempty', 'fof_halo_nonempty']
    print(' | '.join(headers))
    print('-' * 90)
    for r in rows:
        print(' | '.join(str(r[h]) for h in headers))


In [ ]:
if plt is None:
    print('matplotlib not available; skipping plot cell.')
else:
    labels = [r['variant'] for r in rows]
    x = range(len(labels))
    total = [r['fof_dirs_present'] for r in rows]
    center = [r['galcenter_nonempty'] for r in rows]

    plt.figure(figsize=(8, 4))
    plt.bar(x, total, label='FoF dirs present', alpha=0.7)
    plt.bar(x, center, label='GalCenter non-empty', alpha=0.7)
    plt.xticks(list(x), labels)
    plt.ylabel('Snapshot count')
    plt.title('Production Output Coverage')
    plt.legend()
    plt.tight_layout()
    plt.show()
